<a href="https://colab.research.google.com/github/Amik-Sen-Fun/pytorch-tutorial/blob/main/Linear_Regression_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)

Keep this picture in mind as you work through the next few tutorials. 

## Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

In [51]:
import numpy as np
import torch, torchvision

In [52]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [53]:
# The inputs are in numpy array, so we will convert them into tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs, targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]]) tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [54]:
# Generating the initial Random Weight and Bias Matrix

w = torch.randn(2,3, requires_grad = True)  # Generates the random weight matrix
b = torch.randn(2,requires_grad = True)    # Generates the random bias matrix
print(w,b)

# Now let's see the initial guess
y = inputs @ w.t() + b # Here @ -> matrix multiplication and .t() -> transpose
print(y)

tensor([[-0.0522, -0.6296,  1.0169],
        [ 1.5011,  1.4877,  0.9603]], requires_grad=True) tensor([1.9737, 0.3603], requires_grad=True)
tensor([[-2.9486e-01,  2.5091e+02],
        [ 6.8985e+00,  3.2934e+02],
        [-2.7955e+01,  3.8601e+02],
        [ 7.1989e+00,  2.5298e+02],
        [ 9.1124e+00,  3.1398e+02]], grad_fn=<AddBackward0>)


## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [55]:
# The mean square error is computed as 
def mse(t1,t2):
  diff = (t1-t2)
  print(diff*diff)
  print(torch.sum(diff*diff))
  print(diff.numel())
  return torch.sum(diff*diff)/ diff.numel()

# torch.sum() -> gives the sum of square of all errors, diff.numel() -> gives total number of elements
loss = mse(y,targets)
print(loss)

tensor([[ 3169.1113, 32729.0430],
        [ 5491.0278, 52139.2812],
        [21595.6523, 64014.0195],
        [  219.0735, 46645.6562],
        [ 8814.8740, 38017.1211]], grad_fn=<MulBackward0>)
tensor(272834.8750, grad_fn=<SumBackward0>)
10
tensor(27283.4883, grad_fn=<DivBackward0>)


In [56]:
# Computes all the derivatives required
loss.backward()
print(w.grad, b.grad)

tensor([[-6325.1533, -7926.8511, -4561.2622],
        [18296.1094, 18824.6797, 11741.4512]]) tensor([-77.2079, 214.6436])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [57]:
# now lets decrease the weights and bias
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  print(w,b)

tensor([[ 0.0110, -0.5503,  1.0625],
        [ 1.3181,  1.2995,  0.8429]], requires_grad=True) tensor([1.9744, 0.3581], requires_grad=True)


We multiply the gradients with a very small number (`10^-5` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [58]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

Now we will train our model iteratively and try to reduce the loss

In [59]:
for epoch in range(100):
  pred = inputs @ w.t() + b
  loss = mse(pred, targets)
  loss.backward()
  with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5

tensor([[1.9718e+03, 2.2468e+04],
        [3.4164e+03, 3.5197e+04],
        [1.6431e+04, 4.2048e+04],
        [1.0579e+01, 3.4178e+04],
        [6.1968e+03, 2.4356e+04]], grad_fn=<MulBackward0>)
tensor(186272.1875, grad_fn=<SumBackward0>)
10
tensor([[ 1200.8859, 15480.3320],
        [ 2080.3196, 23767.1836],
        [12709.6504, 27460.9434],
        [   38.0380, 25374.7402],
        [ 4388.9028, 15408.9248]], grad_fn=<MulBackward0>)
tensor(127909.9297, grad_fn=<SumBackward0>)
10
tensor([[  285.9203,  6088.6665],
        [  494.6172,  8698.4395],
        [ 7152.7598,  8854.9463],
        [  541.0465, 12695.2471],
        [ 1894.0608,  4357.1343]], grad_fn=<MulBackward0>)
tensor(51062.8398, grad_fn=<SumBackward0>)
10
tensor([[  31.7082,  361.5519],
        [  55.6270,  249.7315],
        [2368.2520,    9.5569],
        [2011.1138, 2837.4844],
        [ 212.2075,   61.8670]], grad_fn=<MulBackward0>)
tensor(8199.0996, grad_fn=<SumBackward0>)
10
tensor([[ 835.1704, 1770.7388],
        [1454

> **NOTE :**  Here the *number of epochs* and *1e-5* are known as hyperparameter and we need to fine tune them in order to obtain desired results

In [60]:
# Finally the results are:
pred = inputs @ w.t() + b
print(f"{pred} and this is the target {targets} and loss is {mse(pred, targets)}")

tensor([[ 544.3773, 3745.7402],
        [1358.2867, 7191.4858],
        [ 495.9643, 8432.7803],
        [ 897.8066, 2885.5598],
        [1540.3140, 7957.9409]], grad_fn=<MulBackward0>)
tensor(35050.2578, grad_fn=<SumBackward0>)
10
tensor([[ 32.6681, 131.2025],
        [ 44.1451, 185.8026],
        [ 96.7297, 224.8302],
        [ -7.9634,  90.7174],
        [ 63.7532, 208.2073]], grad_fn=<AddBackward0>) and this is the target tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]]) and loss is 3505.02587890625


## Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the `torch.nn` package from PyTorch, which contains utility classes for building neural networks.

In [61]:
import torch.nn as nn
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [62]:
from torch.utils.data import TensorDataset

# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]
train_ds[[1,3,5,7]] # To access the 1st, 3rd, 5th and 7th data entry

(tensor([[ 91.,  88.,  64.],
         [102.,  43.,  37.],
         [ 74.,  66.,  43.],
         [ 88., 134.,  59.]]), tensor([[ 81., 101.],
         [ 22.,  37.],
         [ 57.,  69.],
         [118., 132.]]))

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In each iteration, the data loader returns one batch of data with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

In [63]:
from torch.utils.data import DataLoader

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

# Watching the first batch
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  96.,  71.],
        [103.,  43.,  36.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 92.,  87.,  64.]])
tensor([[104., 118.],
        [ 20.,  38.],
        [103., 119.],
        [ 57.,  69.],
        [ 82., 100.]])


## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [64]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

# Parameters use
list(model.parameters())

Parameter containing:
tensor([[-0.3600,  0.0821, -0.4679],
        [ 0.2426,  0.1903,  0.5423]], requires_grad=True)
Parameter containing:
tensor([-0.4796,  0.5256], requires_grad=True)


[Parameter containing:
 tensor([[-0.3600,  0.0821, -0.4679],
         [ 0.2426,  0.1903,  0.5423]], requires_grad=True),
 Parameter containing:
 tensor([-0.4796,  0.5256], requires_grad=True)]

In [65]:
pred = model(inputs)
print(pred)

tensor([[-41.3790,  54.3067],
        [-55.9611,  74.0585],
        [-47.9383,  78.5887],
        [-50.9808,  53.5216],
        [-50.1924,  73.4970],
        [-41.8211,  54.3590],
        [-56.5111,  74.4105],
        [-48.7662,  79.3736],
        [-50.5387,  53.4693],
        [-50.3003,  73.7966],
        [-41.9290,  54.6586],
        [-56.4031,  74.1108],
        [-47.3883,  78.2367],
        [-50.8729,  53.2219],
        [-49.7503,  73.4446]], grad_fn=<AddmmBackward>)


## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

The `nn.functional` package contains many useful loss functions and several other utilities.

In [66]:
# Import nn.functional
import torch.nn.functional as F

# Define loss function
loss_fn = F.mse_loss

# Let's compute the loss for the current predictions of our model.
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(9099.5879, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

> **Note :** that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

In [69]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [70]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [72]:
fit(100, model, loss_fn, opt, train_dl)

# Generate predictions
preds = model(inputs)
print(f"These are the predictions \n{preds}\n These were the targets \n{targets}")

Epoch [10/100], Loss: 11.9604
Epoch [20/100], Loss: 30.3884
Epoch [30/100], Loss: 29.8352
Epoch [40/100], Loss: 20.1526
Epoch [50/100], Loss: 28.6953
Epoch [60/100], Loss: 13.5210
Epoch [70/100], Loss: 17.6451
Epoch [80/100], Loss: 9.8236
Epoch [90/100], Loss: 18.5780
Epoch [100/100], Loss: 9.0264
These are the predictions 
tensor([[ 57.3224,  70.7159],
        [ 79.1944,  99.8261],
        [124.1410, 133.9783],
        [ 23.0548,  38.9556],
        [ 95.5178, 116.6581],
        [ 56.0332,  69.6405],
        [ 78.5093,  99.7847],
        [124.1488, 134.5061],
        [ 24.3441,  40.0310],
        [ 96.1219, 117.6921],
        [ 56.6373,  70.6745],
        [ 77.9052,  98.7507],
        [124.8261, 134.0198],
        [ 22.4507,  37.9216],
        [ 96.8070, 117.7335]], grad_fn=<AddmmBackward>)
 These were the targets 
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.

## Questions for Review

Try answering the following questions to test your understanding of the topics covered in this notebook:

1. What is a linear regression model? Give an example of a problem formulated as a linear regression model.
2. What are input and target variables in a dataset? Give an example.
3. What are weights and biases in a linear regression model?
4. How do you represent tabular data using PyTorch tensors?
5. Why do we create separate matrices for inputs and targets while training a linear regression model?
6. How do you determine the shape of the weights matrix & bias vector given some training data?
7. How do you create randomly initialized weights & biases with a given shape?
8. How is a linear regression model implemented using matrix operations? Explain with an example.
9. How do you generate predictions using a linear regression model?
10. Why are the predictions of a randomly initialized model different from the actual targets?
11. What is a loss function? What does the term “loss” signify?
12. What is mean squared error?
13. Write a function to calculate mean squared using model predictions and actual targets.
14. What happens when you invoke the `.backward` function on the result of the mean squared error loss function?
15. Why is the derivative of the loss w.r.t. the weights matrix itself a matrix? What do its elements represent?
16. How is the derivate of the loss w.r.t. a weight element useful for reducing the loss? Explain with an example.
17. Suppose the derivative  of the loss w.r.t. a weight element is positive. Should you increase or decrease the element’s value slightly to get a lower loss?
18. Suppose the derivative  of the loss w.r.t. a weight element is negative. Should you increase or decrease the element’s value slightly to get a lower loss?
19. How do you update the weights and biases of a model using their respective gradients to reduce the loss slightly?
20. What is the gradient descent optimization algorithm? Why is it called “gradient descent”?
21. Why do you subtract a “small quantity” proportional to the gradient from the weights & biases, not the actual gradient itself?
22. What is learning rate? Why is it important?
23. What is `torch.no_grad`?
24. Why do you reset gradients to zero after updating weights and biases?
25. What are the steps involved in training a linear regression model using gradient descent?
26. What is an epoch?
27. What is the benefit of training a model for multiple epochs?
28. How do you make predictions using a trained model?
29. What should you do if your model’s loss doesn’t decrease while training? Hint: learning rate.
30. What is `torch.nn`?
31. What is the purpose of the `TensorDataset` class in PyTorch? Give an example.
32. What is a data loader in PyTorch? Give an example.
33. How do you use a data loader to retrieve batches of data?
34. What are the benefits of shuffling the training data before creating batches?
35. What is the benefit of training in small batches instead of training with the entire dataset?
36. What is the purpose of the `nn.Linear` class in PyTorch? Give an example.
37. How do you see the weights and biases of a `nn.Linear` model?
38. What is the purpose of the `torch.nn.functional` module?
39. How do you compute mean squared error loss using a PyTorch built-in function?
40. What is an optimizer in PyTorch?
41. What is `torch.optim.SGD`? What does SGD stand for?
42. What are the inputs to a PyTorch optimizer? 
43. Give an example of creating an optimizer for training a linear regression model.
44. Write a function to train a `nn.Linear` model in batches using gradient descent.
45. How do you use a linear regression model to make predictions on previously unseen data?

## Further Reading

* An visual & animated explanation of gradient descent: https://www.youtube.com/watch?v=IHZwWFHWa-w

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf). 

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.